# Numeric Orbit Propagation

This notebook numerically propagates the trajectory of a spacecraft in a central gravitational field, with an optional third-body perturbation. It sets up orbital initial conditions, integrates the equations of motion with SciPy’s `solve_ivp`, and visualises the resulting orbit in 2D, 3D, and as an interactive animation.

## 1. Setting up the environment

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp  # For solving ODEs
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting
import time  # To measure computation time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as colors

## 2. Constants and initial conditions

This section defines the physical constants and initial orbital state for the propagation, including:

- Gravitational parameters (e.g. Sun, third body)
- Astronomical unit and other scale factors
- Flags to control whether third-body perturbations are included
- The spacecraft’s initial orbital elements (semi-major axis, eccentricity, inclination, RAAN, argument of perigee, true anomaly)

These are stored in small dictionaries to keep the configuration clear and easy to modify.


In [ ]:
# Constants and Conversion Factors
DEG2RAD = np.pi / 180  # Conversion factor from degrees to radians

# Auxiliary structure to pass data to the ODE
auxv = {
    'au': 1.496e11,            # Astronomical unit in meters
    'c': 2.998e8,              # Speed of light in m/s
    'rSun': 696340e3,          # Sun radius in meters (for graphics)
    'muSun': 1.32712440018e20, # Gravitational parameter of the Sun in m^3/s^2
    'muX': 1e19,               # Fictitious gravitational parameter for the third body
    'tbr_flag': True,          # Flag to include third-body perturbation
    'tbr': np.array([1.496e11, 1.496e11, 0.0])  # Position of the third body in meters
}

# Extract tbr_flag and third_body from auxv
tbr_flag = auxv.get('tbr_flag', False)
third_body = auxv['tbr'] if tbr_flag else None

# Spacecraft Keplerian Elements (Assuming Circular Orbit for Simplicity)
kepler_elements = {
    'SMA': 1.0 * auxv['au'],  # Semi-Major Axis in meters (circular orbit)
    'ecc': 0.0,                # Eccentricity (0 for circular orbit)
    'inc': 0.0 * DEG2RAD,      # Inclination in radians
    'AoP': 0.0 * DEG2RAD,      # Argument of Perigee in radians
    'RAAN': 0.0 * DEG2RAD,     # Right Ascension of Ascending Node in radians
    'TA': 0.0 * DEG2RAD        # True Anomaly in radians
}

# Simulation Settings
propagation_time_days = 500        # Propagation time in days
propagation_step_seconds = 5      # Integration step in seconds

# Convert propagation time to seconds
t_final = propagation_time_days * 24 * 60 * 60  # Total simulation time in seconds
t_step = propagation_step_seconds            # Integration step in seconds

## 3. Auxiliary functions

The helper functions in this section:

- Convert between orbital elements and Cartesian state vectors
- Evaluate the equations of motion for the ODE solver
- Apply the central body gravity and optional third-body perturbation

Having these utilities explicitly defined keeps the numerical procedure transparent. In a larger project one could also rely on specialised libraries such as `astropy` or `poliastro`, but here the core operations are implemented directly for clarity.


In [ ]:
# Converting Keplerian Elements to ECI State Vector
def e_oe2ECI(mu, SMA, ecc, inc, AoP, RAAN, TA):
    """
    Converts Keplerian orbital elements to ECI (Earth-Centered Inertial) state vectors.
    
    Parameters:
    - mu: Gravitational parameter [m^3/s^2]
    - SMA: Semi-Major Axis [m]
    - ecc: Eccentricity
    - inc: Inclination [rad]
    - AoP: Argument of Perigee [rad]
    - RAAN: Right Ascension of Ascending Node [rad]
    - TA: True Anomaly [rad]
    
    Returns:
    - state_vector: numpy array containing position (m) and velocity (m/s) vectors
    """
    slr = SMA * (1 - ecc**2)  # Semi-latus rectum (p)
    h = np.sqrt(mu * slr)     # Specific angular momentum (h)
    rm = slr / (1 + ecc * np.cos(TA))  # Radius magnitude at True Anomaly

    # Position in orbital plane
    r_orbital = np.array([
        rm * np.cos(TA),
        rm * np.sin(TA),
        0.0
    ])

    # Velocity in orbital plane
    v_r = (mu / h) * ecc * np.sin(TA)
    v_theta = (mu / h) * (1 + ecc * np.cos(TA))
    v_orbital = np.array([
        v_r * np.cos(TA) - v_theta * np.sin(TA),
        v_r * np.sin(TA) + v_theta * np.cos(TA),
        0.0
    ])

    # Rotation matrices
    R_z_RAAN = rotation_matrix_z(RAAN)
    R_x_inc = rotation_matrix_x(inc)
    R_z_AoP = rotation_matrix_z(AoP)

    # Combined rotation matrix
    rotation_matrix = R_z_RAAN @ R_x_inc @ R_z_AoP

    # Position and velocity in ECI frame
    r_ECI = rotation_matrix @ r_orbital
    v_ECI = rotation_matrix @ v_orbital

    # Combine into a single state vector
    state_vector = np.hstack((r_ECI, v_ECI))
    return state_vector

# Supporting Rotation Matrices
def rotation_matrix_z(angle_rad):
    """
    Creates a rotation matrix for a rotation around the Z-axis by a given angle.
    """
    c = np.cos(angle_rad)
    s = np.sin(angle_rad)
    return np.array([
        [c, -s, 0],
        [s,  c, 0],
        [0,  0, 1]
    ])

def rotation_matrix_x(angle_rad):
    """
    Creates a rotation matrix for a rotation around the X-axis by a given angle.
    """
    c = np.cos(angle_rad)
    s = np.sin(angle_rad)
    return np.array([
        [1, 0,  0],
        [0, c, -s],
        [0, s,  c]
    ])

# Computing Third-Body Perturbations (Point Mass Model)
def g_xbody(mu_body, r_probe, r_body):
    """
    Computes the gravitational acceleration on the probe due to a third body using the point mass model.
    
    Parameters:
    - mu_body: Gravitational parameter of the third body [m^3/s^2]
    - r_probe: Position vector of the probe [m] (numpy array)
    - r_body: Position vector of the third body [m] (numpy array)
    
    Returns:
    - ag_body: Gravitational acceleration vector [m/s^2] due to the third body
    """
    # Vector from probe to third body
    r_probe_to_body = r_body - r_probe
    distance = np.linalg.norm(r_probe_to_body)
    if distance == 0:
        return np.array([0.0, 0.0, 0.0])
    ag_body = mu_body * r_probe_to_body / distance**3
    return ag_body

# Main ODE Function
def orbit_ode(t, posvel, auxv):
    """
    Defines the ODE system for orbital propagation.
    
    Parameters:
    - t: Time variable (not used as the system is time-invariant)
    - posvel: State vector [x, y, z, vx, vy, vz]
    - auxv: Dictionary containing auxiliary parameters
    
    Returns:
    - dy_dt: Derivative of state vector
    """
    # Unpack position and velocity
    r = posvel[0:3]  # Position vector [x, y, z]
    v = posvel[3:6]  # Velocity vector [vx, vy, vz]
    
    # Compute magnitude of position and velocity
    r_mag = np.linalg.norm(r)  # Magnitude of position vector
    v_mag = np.linalg.norm(v)  # Magnitude of velocity vector
    
    # Gravitational acceleration due to the Sun
    a_sun = -auxv['muSun'] * r / r_mag**3
    
    # Post-Newtonian Relativistic Correction (Optional)
    a_rel = -(auxv['muSun'] / (r_mag**2)) * (
        (4 * (auxv['muSun'] / (auxv['c']**2 * r_mag)) - (v_mag**2) / (auxv['c']**2)) * (r / r_mag)
        + (4 / auxv['c']**2) * np.dot(r, v) / (r_mag**2) * v
    )
    
    # Initialize total acceleration with gravitational and relativistic terms
    a_total = a_sun + a_rel
    
    # Include third-body perturbation if flagged
    if auxv['tbr_flag']:
        a_thb = g_xbody(auxv['muX'], r, auxv['tbr'])
        a_total += a_thb
    
    # Assemble the derivative of the state vector
    dy_dt = np.zeros(6)
    dy_dt[0:3] = v  # Derivative of position is velocity
    dy_dt[3:6] = a_total  # Derivative of velocity is acceleration
    
    return dy_dt

## 4. Numerical integration with `solve_ivp`

Here we define the time span for the simulation and call SciPy’s `solve_ivp` to integrate the equations of motion. The result is a time history of the spacecraft’s Cartesian position and velocity which we can then analyse and visualise.


In [ ]:
# Convert Keplerian elements to ECI state vector
ECI_SV_0 = e_oe2ECI(
    mu=auxv['muSun'],
    SMA=kepler_elements['SMA'],
    ecc=kepler_elements['ecc'],
    inc=kepler_elements['inc'],
    AoP=kepler_elements['AoP'],
    RAAN=kepler_elements['RAAN'],
    TA=kepler_elements['TA']
)

# Define the time span for the integration
t_span = (0, t_final)  # From 0 to t_final seconds

# Start timer
start_time = time.time()

# Run the ODE solver
solution = solve_ivp(
    fun=lambda t, y: orbit_ode(t, y, auxv),
    t_span=t_span,
    y0=ECI_SV_0,
    method='RK45',
    rtol=1e-6,
    atol=1e-6
)

# End timer
end_time = time.time()
print(f"Numeric propagation computed in {end_time - start_time:.2f} seconds")

## 5. Visualising the propagated orbit

We now generate plots of the propagated trajectory to inspect the orbit:

- A 3D view of the trajectory in the inertial frame
- 2D projections to make the overall shape and scale easier to interpret

These visualisations provide a quick sanity check on the propagation and the chosen initial conditions.


In [ ]:
# Extract time and state vectors from the solution
t = solution.t  # Time in seconds
PosVel = solution.y.T  # Transpose to have shape (N, 6)

# Calculate magnitudes of position and velocity
r_mag = np.linalg.norm(PosVel[:, 0:3], axis=1)  # Position magnitude [m]
vmag = np.linalg.norm(PosVel[:, 3:6], axis=1)   # Velocity magnitude [m/s]

# Indices for plotting
ti = 0  # Start index
tf = len(t) - 1  # End index

# Matplotlib 2D Plotting
fig, ax1 = plt.subplots(figsize=(12, 6), facecolor='white')
fig.suptitle('STATE VECTOR: ORBIT1', fontsize=16)
ax1.set_xlabel('Time [seconds]', fontsize=14)

# Plot SC Position |r| on the left y-axis
color = 'tab:blue'
ax1.set_ylabel('SC Position |r| [m]', color=color, fontsize=14)
ax1.plot(t, r_mag, color=color, linewidth=2, label='Position |r|')
ax1.tick_params(axis='y', labelcolor=color)

# Create a twin y-axis to plot SC Speed |v|
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('SC Speed |v| [m/s]', color=color, fontsize=14)
ax2.plot(t, vmag, color=color, linewidth=2, label='Speed |v|')
ax2.tick_params(axis='y', labelcolor=color)

# Optional: Add legends
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

# Improve layout and display the plot
fig.tight_layout()  # Adjust subplots to fit into figure area.
fig.subplots_adjust(top=0.92)  # Adjust the top to make room for the suptitle
plt.show()

### 3D orbit plot

This plot shows the spacecraft trajectory in 3D Cartesian coordinates, together with the central body at the origin. It is useful for understanding the overall geometry of the orbit and any perturbation effects.


In [ ]:
# Function to generate sphere vertices (for Sun, Earth, Third Body)
def create_sphere(center, radius, u_steps=50, v_steps=25):
    u, v = np.mgrid[0:2*np.pi:u_steps*1j, 0:np.pi:v_steps*1j]
    x = center[0] + radius * np.cos(u) * np.sin(v)
    y = center[1] + radius * np.sin(u) * np.sin(v)
    z = center[2] + radius * np.cos(v)
    return x, y, z

# Matplotlib 3D Plotting
fig_3d = plt.figure(figsize=(12, 6), facecolor='gray')
ax_3d = fig_3d.add_subplot(111, projection='3d')
ax_3d.set_title('ORBIT 3D VIEW', fontsize=16)

# Plot the Sun as a sphere
sun_x, sun_y, sun_z = create_sphere([0, 0, 0], auxv['rSun'])
ax_3d.plot_surface(sun_x, sun_y, sun_z, color='yellow', edgecolor='none', alpha=0.5)

# Plot coordinate axes
axis_length = 0.5 * auxv['au']
ax_3d.plot([0, axis_length], [0, 0], [0, 0], color='red', linewidth=2)  # X-axis
ax_3d.plot([0, 0], [0, axis_length], [0, 0], color='green', linewidth=2)  # Y-axis
ax_3d.plot([0, 0], [0, 0], [0, axis_length], color='blue', linewidth=2)  # Z-axis

# Plot the spacecraft's trajectory
ax_3d.plot(PosVel[:, 0], PosVel[:, 1], PosVel[:, 2], color='cyan', linestyle=':', linewidth=2)

# Plot the spacecraft's final position as a sphere
final_pos = PosVel[-1, 0:3]
r_sc = 5 * auxv['rSun']  # Scaling factor for visualization
X_sc, Y_sc, Z_sc = create_sphere(final_pos, r_sc)
ax_3d.plot_surface(
    X_sc,
    Y_sc,
    Z_sc,
    color='cyan', edgecolor='none', alpha=0.6
)

# Plot the third body if flagged
if tbr_flag and third_body is not None:
    tb_x, tb_y, tb_z = create_sphere(third_body, auxv['rSun'] * 5)
    ax_3d.plot_surface(
        tb_x,
        tb_y,
        tb_z,
        color='red', edgecolor='none', alpha=0.6
    )

# Set equal aspect ratio
max_val = np.max(np.abs(PosVel[:, 0:3]))
ax_3d.set_xlim([-max_val, max_val])
ax_3d.set_ylim([-max_val, max_val])
ax_3d.set_zlim([-max_val, max_val])
ax_3d.set_box_aspect([1,1,1])  # Equal aspect ratio

plt.show()

## 6. Interactive animation

Finally, an interactive Plotly animation of the orbit is generated. You can rotate, zoom, and scrub through time to explore the spacecraft motion from different viewpoints and at different points along the trajectory.


In [ ]:
# Extract dynamic position components
x = PosVel[:, 0]
y = PosVel[:, 1]
z = PosVel[:, 2]

# Define the number of frames and frame indices
num_frames = 100  # Desired number of frames
total_time_points = len(t)

# Ensure that total_time_points >= num_frames
if total_time_points < num_frames:
    print("Error: Not enough time points to generate the desired number of frames.")
    num_frames = total_time_points
    print(f"Adjusted number of frames to {num_frames}.")

# Calculate frame indices using numpy's linspace
idx_values = np.linspace(0, total_time_points - 1, num_frames, dtype=int)
print(f"Frame indices: {idx_values}")

# Sun
sun_x, sun_y, sun_z = create_sphere([0, 0, 0], auxv['rSun'] * 2)  # Increased radius for visibility

# Earth (optional)
earth_x, earth_y, earth_z = create_sphere([auxv['au'], 0, 0], 6371e3)  # Earth's position and radius

# Third Body (if applicable)
if tbr_flag and third_body is not None:
    tb_x, tb_y, tb_z = create_sphere(third_body, auxv['rSun'] * 5)
    third_body_trace = go.Surface(
        x=tb_x,
        y=tb_y,
        z=tb_z,
        colorscale=[[0, 'red'], [1, 'red']],
        showscale=False,
        name='Third Body',
        hoverinfo='skip',
        opacity=0.6
    )
else:
    third_body_trace = None

# Generate frames
frames = []
for idx in idx_values:
    # Ensure idx is within bounds
    if idx >= total_time_points:
        idx = total_time_points - 1

    # Define dynamic traces' data
    trajectory = go.Scatter3d(
        x=x[:idx+1],
        y=y[:idx+1],
        z=z[:idx+1],
        mode='lines',
        line=dict(color='cyan', width=4),
        name='Spacecraft Trajectory',
        hoverinfo='skip'
    )

    position = go.Scatter3d(
        x=[x[idx]],
        y=[y[idx]],
        z=[z[idx]],
        mode='markers',
        marker=dict(size=6, color='cyan', symbol='circle'),
        name='Spacecraft Position',
        hoverinfo='skip'
    )

    # Define static traces' data without stars
    static_traces = [
        go.Surface(
            x=sun_x,
            y=sun_y,
            z=sun_z,
            colorscale=[[0, 'yellow'], [1, 'yellow']],
            showscale=False,
            name='Sun',
            hoverinfo='skip',
            opacity=1.0
        ),
        go.Surface(
            x=earth_x,
            y=earth_y,
            z=earth_z,
            colorscale=[[0, 'blue'], [1, 'green']],
            showscale=False,
            name='Earth',
            hoverinfo='skip',
            opacity=0.6
        )
    ]

    # Add Third Body if applicable
    if third_body_trace is not None:
        static_traces.append(third_body_trace)

    # Compile all traces for the frame
    frame_data = static_traces + [trajectory, position]

    # Create the frame
    frame = go.Frame(
        data=frame_data,
        name=str(idx)  # Frame names correspond to indices
    )

    frames.append(frame)

print(f"Total frames generated: {len(frames)}")  # Should print 100

# Start timer
start_time = time.time()

# Create a new figure
fig = go.Figure()

# 1. Plot the Sun as a bright yellow sphere
fig.add_trace(go.Surface(
    x=sun_x,
    y=sun_y,
    z=sun_z,
    colorscale=[[0, 'yellow'], [1, 'yellow']],
    showscale=False,
    name='Sun',
    hoverinfo='skip',
    opacity=1.0  # Full opacity
))

# 2. Plot the Earth as a blue-green sphere (optional)
fig.add_trace(go.Surface(
    x=earth_x,
    y=earth_y,
    z=earth_z,
    colorscale=[[0, 'blue'], [1, 'green']],
    showscale=False,
    name='Earth',
    hoverinfo='skip',
    opacity=0.6
))

# 3. Plot the initial spacecraft trajectory
fig.add_trace(go.Scatter3d(
    x=[x[0]],
    y=[y[0]],
    z=[z[0]],
    mode='lines',
    name='Spacecraft Trajectory',
    line=dict(color='cyan', width=4),
    hoverinfo='skip'
))

# 4. Plot the initial spacecraft position
fig.add_trace(go.Scatter3d(
    x=[x[0]],
    y=[y[0]],
    z=[z[0]],
    mode='markers',
    name='Spacecraft Position',
    marker=dict(size=6, color='cyan', symbol='circle'),
    hoverinfo='skip'
))

# 5. Plot the third body as a red sphere (if flagged)
if tbr_flag and third_body is not None:
    fig.add_trace(go.Surface(
        x=tb_x,
        y=tb_y,
        z=tb_z,
        colorscale=[[0, 'red'], [1, 'red']],
        showscale=False,
        name='Third Body',
        hoverinfo='skip',
        opacity=0.6
    ))

# 6. Plot coordinate axes for reference
axis_length = 0.5 * auxv['au']
fig.add_trace(go.Scatter3d(
    x=[0, axis_length],
    y=[0, 0],
    z=[0, 0],
    mode='lines',
    line=dict(color='white', width=4),
    name='X-axis'
))
fig.add_trace(go.Scatter3d(
    x=[0, 0],
    y=[0, axis_length],
    z=[0, 0],
    mode='lines',
    line=dict(color='white', width=4),
    name='Y-axis'
))
fig.add_trace(go.Scatter3d(
    x=[0, 0],
    y=[0, 0],
    z=[0, axis_length],
    mode='lines',
    line=dict(color='white', width=4),
    name='Z-axis'
))

# Assign the frames to the figure
fig.frames = frames

# Define Play and Pause buttons
play_pause_buttons = [
    dict(
        label='Play',
        method='animate',
        args=[
            None,
            dict(
                frame=dict(duration=50, redraw=True),
                transition=dict(duration=0),
                fromcurrent=True,
                mode='immediate'
            )
        ]
    ),
    dict(
        label='Pause',
        method='animate',
        args=[
            [None],
            dict(
                frame=dict(duration=0, redraw=False),
                transition=dict(duration=0),
                mode='immediate'
            )
        ]
    )
]

# Define Slider Steps
slider_steps = []
for idx in idx_values:
    step = dict(
        method='animate',
        args=[
            [str(idx)],  # Frame name as string
            dict(
                mode='immediate',
                frame=dict(duration=50, redraw=True),
                transition=dict(duration=0)
            )
        ],
        label=f'{t[int(idx)]/86400:.1f} days'  # Label shows time in days
    )
    slider_steps.append(step)

# Define Sliders
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time: "},
    pad={"t": 50},
    steps=slider_steps,
    len=1.0,
    x=0,
    y=0
)]

# Update layout with animation controls and aesthetic enhancements
fig.update_layout(
    title='Interactive 3D Orbit Simulation with Animation',
    scene=dict(
        xaxis=dict(
            title='X [m]',
            backgroundcolor="black",
            gridcolor="gray",
            showbackground=True,
            zerolinecolor="gray",
            showticklabels=False,
            ticks=''
        ),
        yaxis=dict(
            title='Y [m]',
            backgroundcolor="black",
            gridcolor="gray",
            showbackground=True,
            zerolinecolor="gray",
            showticklabels=False,
            ticks=''
        ),
        zaxis=dict(
            title='Z [m]',
            backgroundcolor="black",
            gridcolor="gray",
            showbackground=True,
            zerolinecolor="gray",
            showticklabels=False,
            ticks=''
        ),
        aspectmode='data',  # Ensures equal scaling on all axes
        camera=dict(
            eye=dict(x=1.25, y=1.25, z=1.25)  # Adjust camera position if needed
        )
    ),
    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            y=1,
            x=1.05,
            xanchor='right',
            yanchor='top',
            pad=dict(t=0, r=10),
            buttons=play_pause_buttons
        )
    ],
    sliders=sliders,
    width=1000,
    height=900,
    paper_bgcolor='black',
    plot_bgcolor='black',
    legend=dict(
        x=0.7,
        y=0.95,
        bgcolor='rgba(0,0,0,0.5)',
        bordercolor='White',
        borderwidth=1
    )
)

# Display the interactive plot
fig.show()

# End timer
end_time = time.time()
print(f"Graphical representations computed in {end_time - start_time:.2f} seconds")

## 7. Summary and possible extensions

- This notebook sets up and propagates a spacecraft orbit numerically using SciPy’s `solve_ivp`.
- The gravitational environment, initial orbital elements and integration settings are all configurable.
- Static and interactive visualisations are provided to inspect the resulting trajectory.

Possible extensions include:
- Adding more detailed perturbation models (e.g. J2, solar radiation pressure)
- Comparing different numerical integrators or step sizes
- Wrapping the propagation in reusable functions/classes for larger mission analysis studies.
